# Examing the effects of dynamic creators

In [7]:
from trecs.components import Creators, Users, Items
from trecs.models import ContentFiltering, PopularityRecommender
import numpy as np

In [8]:
# parent directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils import create_profiles, calc_group_preferences
from dynamic_creators import DynamicCreators
from custom_metrics import CreatorAvgGroupSkew

In [9]:
user_profiles, item_profiles, creator_profiles = create_profiles(
    total_users = 100,
    num_majority_users = 90,
    total_items = 10,
    num_majority_items = 1,
    total_creators = 50,
    dynamic_creators = True,
    num_attrs = 7,
    separate_attr = True,
    group_strength= 5.0
)

Percentage of items generated by Group A: 0.1


In [10]:
print(user_profiles.shape, item_profiles.shape, creator_profiles.actual_creator_profiles.shape)

(100, 7) (7, 10) (50, 6)


In [11]:
creator_profiles.learning_rate = 0.1
cf = ContentFiltering(
    actual_item_representation=item_profiles,
    item_representation=item_profiles,
    actual_user_representation=Users(actual_user_profiles=user_profiles),
    creators = creator_profiles,
    num_items_per_iter=10
)
cf.add_metrics(CreatorAvgGroupSkew())
# randomly interleave as many as 10 random items per timestep
cf.run(timesteps=300, random_items_per_iter=10, vary_random_items_per_iter=True)

100%|██████████| 300/300 [00:12<00:00, 23.94it/s]


In [12]:
np.array(cf.get_measurements()['creator_group_skew'][1:]).mean(axis=1)

array([0.50026786, 0.5069085 , 0.53128441, 0.53663441, 0.5492216 ,
       0.56225323, 0.56225323, 0.56430095, 0.55092984, 0.55990135,
       0.55990135, 0.55990135, 0.55990135, 0.55990135, 0.55990135,
       0.55990135, 0.55990135, 0.56067482, 0.56067482, 0.56067482,
       0.56067482, 0.56067482, 0.56067482, 0.56067482, 0.56067482,
       0.56067482, 0.56067482, 0.56067482, 0.56067482, 0.55867482,
       0.55867482, 0.55667482, 0.55667482, 0.55667482, 0.5523609 ,
       0.5523609 , 0.5523609 , 0.5523609 , 0.552154  , 0.552154  ,
       0.552154  , 0.552154  , 0.552154  , 0.552154  , 0.552154  ,
       0.56680635, 0.5682252 , 0.5682252 , 0.5682252 , 0.5682252 ,
       0.5682252 , 0.5682252 , 0.5682252 , 0.5682252 , 0.5742252 ,
       0.5742252 , 0.585805  , 0.585805  , 0.585805  , 0.585805  ,
       0.585805  , 0.585805  , 0.585805  , 0.585805  , 0.585805  ,
       0.585805  , 0.585805  , 0.585805  , 0.585805  , 0.585805  ,
       0.585805  , 0.585805  , 0.585805  , 0.585805  , 0.58580